# Router Agent - Test Notebook

This notebook tests the Router Agent in isolation.

## Purpose
- Test question classification (needle vs summary)
- Verify routing logic
- Test edge cases and borderline questions
- Human inspection of routing decisions

## What This Tests
✅ Needle question classification  
✅ Summary question classification  
✅ Confidence scoring  
✅ Reasoning quality  
✅ Edge cases and borderline questions  
✅ Structured output format  

## What This Does NOT Test
❌ Data retrieval (Router doesn't retrieve)  
❌ Answer generation (Router doesn't answer)  
❌ FAISS/embeddings (Router doesn't touch data)  
❌ Claim-specific logic (Router is claim-agnostic)

**CRITICAL:** This agent ONLY classifies questions. It never retrieves data or generates answers.


---
## Setup


In [2]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

print(f"🐍 Python executable: {sys.executable}")
print(f"🐍 Python version: {sys.version.split()[0]}")
print(f"\n📁 Project root: {project_root}")

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Check for OpenAI API key
if "OPENAI_API_KEY" not in os.environ:
    print("\n⚠️  WARNING: OPENAI_API_KEY not found in environment")
    print("   Set it with: export OPENAI_API_KEY='your-key'")
    print("   Or create a .env file in project root")
else:
    key = os.environ["OPENAI_API_KEY"]
    print(f"\n✅ OPENAI_API_KEY loaded from .env file")
    print(f"   Key starts with: {key[:10]}...")


🐍 Python executable: /opt/anaconda3/envs/ragagent/bin/python
🐍 Python version: 3.11.14

📁 Project root: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2

✅ OPENAI_API_KEY loaded from .env file
   Key starts with: sk-proj-vb...


In [3]:
# Import Router Agent
from RAG.Agents import RouterAgent

print("✅ Router Agent imported successfully")


✅ Router Agent imported successfully


---
## Initialize Router Agent


In [4]:
# Create Router Agent instance
router = RouterAgent(
    model="gpt-4o-mini",  # Fast and cheap for classification
    temperature=0.0        # Deterministic routing
)

print("\n🎯 Router Agent is ready for classification!")


✅ Router Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.0
   Output: Structured (Pydantic)

🎯 Router Agent is ready for classification!


---
## Test 1: Clear NEEDLE Questions

These questions ask for specific, atomic facts.


In [5]:
print("🎯 Testing NEEDLE Questions")
print("=" * 70)
print("\nNeedle questions ask for specific, atomic facts.\n")

needle_questions = [
    "What is the claimant's phone number?",
    "When did the accident occur?",
    "What is the VIN number?",
    "How much was the repair estimate?",
    "Who is the insurance adjuster assigned to this claim?",
]

for i, question in enumerate(needle_questions, 1):
    print(f"\n{'─' * 70}")
    print(f"Question {i}: {question}")
    print(f"{'─' * 70}")
    
    result = router.route(question)
    
    print(f"Route:      {result['route'].upper()}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reason:     {result['reason']}")
    
    # Validation
    if result['route'] == 'needle':
        print("✅ CORRECT - Routed to NEEDLE")
    else:
        print("❌ INCORRECT - Should be NEEDLE")


🎯 Testing NEEDLE Questions

Needle questions ask for specific, atomic facts.


──────────────────────────────────────────────────────────────────────
Question 1: What is the claimant's phone number?
──────────────────────────────────────────────────────────────────────
Route:      NEEDLE
Confidence: 1.00
Reason:     The question asks for a specific piece of information, which is the claimant's phone number, making it a clear case for the NEEDLE route.
✅ CORRECT - Routed to NEEDLE

──────────────────────────────────────────────────────────────────────
Question 2: When did the accident occur?
──────────────────────────────────────────────────────────────────────
Route:      NEEDLE
Confidence: 1.00
Reason:     The question asks for a specific fact regarding the date of the accident, which can be answered with one atomic piece of information.
✅ CORRECT - Routed to NEEDLE

──────────────────────────────────────────────────────────────────────
Question 3: What is the VIN number?
────────────

---
## Test 2: Clear SUMMARY Questions

These questions require contextual, multi-fact answers.


In [6]:
print("📚 Testing SUMMARY Questions")
print("=" * 70)
print("\nSummary questions require contextual, multi-fact answers.\n")

summary_questions = [
    "Summarize the incident that led to this claim.",
    "Describe what happened during the accident.",
    "Explain the damages reported in the claim.",
    "Tell me about the circumstances surrounding the collision.",
    "What were the sequence of events that caused the damage?",
]

for i, question in enumerate(summary_questions, 1):
    print(f"\n{'─' * 70}")
    print(f"Question {i}: {question}")
    print(f"{'─' * 70}")
    
    result = router.route(question)
    
    print(f"Route:      {result['route'].upper()}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reason:     {result['reason']}")
    
    # Validation
    if result['route'] == 'summary':
        print("✅ CORRECT - Routed to SUMMARY")
    else:
        print("❌ INCORRECT - Should be SUMMARY")


📚 Testing SUMMARY Questions

Summary questions require contextual, multi-fact answers.


──────────────────────────────────────────────────────────────────────
Question 1: Summarize the incident that led to this claim.
──────────────────────────────────────────────────────────────────────
Route:      SUMMARY
Confidence: 1.00
Reason:     The question asks for a summary of the incident, which requires multiple facts and contextual understanding rather than a single atomic piece of information.
✅ CORRECT - Routed to SUMMARY

──────────────────────────────────────────────────────────────────────
Question 2: Describe what happened during the accident.
──────────────────────────────────────────────────────────────────────
Route:      SUMMARY
Confidence: 1.00
Reason:     The question asks for an explanation of events, requiring multiple facts and contextual understanding about the accident.
✅ CORRECT - Routed to SUMMARY

──────────────────────────────────────────────────────────────────────
Q

---
## Test 3: Borderline / Edge Cases

These questions are ambiguous or could go either way.


In [7]:
print("⚖️  Testing BORDERLINE / EDGE CASE Questions")
print("=" * 70)
print("\nThese questions are ambiguous or could go either way.\n")
print("Expected: Should default to SUMMARY for safety (more complete answers).\n")

borderline_questions = [
    "What are the damages?",  # Could be list (needle) or description (summary)
    "Tell me about the claim.",  # Vague - summary safer
    "What happened?",  # Very broad - summary
    "Where did it occur?",  # Could be address (needle) or description (summary)
    "What was involved in the accident?",  # Could be specific or contextual
]

for i, question in enumerate(borderline_questions, 1):
    print(f"\n{'─' * 70}")
    print(f"Question {i}: {question}")
    print(f"{'─' * 70}")
    
    result = router.route(question)
    
    print(f"Route:      {result['route'].upper()}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reason:     {result['reason']}")
    
    # Note: For borderline cases, we prefer SUMMARY (safer)
    if result['route'] == 'summary':
        print("✅ Routed to SUMMARY (safer for ambiguous questions)")
    else:
        print("⚠️  Routed to NEEDLE (acceptable if confidence is high)")


⚖️  Testing BORDERLINE / EDGE CASE Questions

These questions are ambiguous or could go either way.

Expected: Should default to SUMMARY for safety (more complete answers).


──────────────────────────────────────────────────────────────────────
Question 1: What are the damages?
──────────────────────────────────────────────────────────────────────
Route:      SUMMARY
Confidence: 0.90
Reason:     The question asks for an overview of the damages, which likely requires multiple facts or context to answer adequately.
✅ Routed to SUMMARY (safer for ambiguous questions)

──────────────────────────────────────────────────────────────────────
Question 2: Tell me about the claim.
──────────────────────────────────────────────────────────────────────
Route:      SUMMARY
Confidence: 1.00
Reason:     The question asks for an overview or narrative about the claim, which requires multiple facts or contextual understanding.
✅ Routed to SUMMARY (safer for ambiguous questions)

───────────────────────

---
## Test 4: Batch Processing

Test routing multiple questions at once.


In [8]:
print("📦 Testing BATCH Processing")
print("=" * 70)
print("\nRouting multiple questions at once.\n")

batch_questions = [
    "What is the policy number?",  # Needle
    "Describe the accident.",  # Summary
    "How much is the claim total?",  # Needle
]

results = router.route_batch(batch_questions)

for i, (question, result) in enumerate(zip(batch_questions, results), 1):
    print(f"\n{'─' * 70}")
    print(f"Question {i}: {question}")
    print(f"Route:      {result['route'].upper()}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reason:     {result['reason']}")

print("\n✅ Batch processing works!")


📦 Testing BATCH Processing

Routing multiple questions at once.


──────────────────────────────────────────────────────────────────────
Question 1: What is the policy number?
Route:      NEEDLE
Confidence: 1.00
Reason:     The question asks for a specific identifier, which can be answered with one atomic piece of information.

──────────────────────────────────────────────────────────────────────
Question 2: Describe the accident.
Route:      SUMMARY
Confidence: 1.00
Reason:     The question asks for a description of the accident, which requires multiple facts and contextual understanding.

──────────────────────────────────────────────────────────────────────
Question 3: How much is the claim total?
Route:      NEEDLE
Confidence: 1.00
Reason:     The question asks for a specific value related to the claim total, which can be answered with one atomic piece of information.

✅ Batch processing works!


---
## Test 5: Confidence Analysis

Analyze confidence scores across all tests.


In [9]:
print("📊 Confidence Score Analysis")
print("=" * 70)

# Collect all test questions
all_questions = {
    "Needle": needle_questions,
    "Summary": summary_questions,
    "Borderline": borderline_questions
}

for category, questions in all_questions.items():
    print(f"\n{category} Questions:")
    
    confidences = []
    for q in questions:
        result = router.route(q)
        confidences.append(result['confidence'])
    
    avg_confidence = sum(confidences) / len(confidences)
    min_confidence = min(confidences)
    max_confidence = max(confidences)
    
    print(f"  Average confidence: {avg_confidence:.3f}")
    print(f"  Min confidence:     {min_confidence:.3f}")
    print(f"  Max confidence:     {max_confidence:.3f}")
    print(f"  Total questions:    {len(questions)}")

print("\n" + "=" * 70)
print("📌 Confidence Interpretation:")
print("  0.9 - 1.0: Very confident (clear classification)")
print("  0.7 - 0.9: Confident (good classification)")
print("  0.5 - 0.7: Moderate (borderline case)")
print("  Below 0.5: Low (ambiguous question)")


📊 Confidence Score Analysis

Needle Questions:
  Average confidence: 1.000
  Min confidence:     1.000
  Max confidence:     1.000
  Total questions:    5

Summary Questions:
  Average confidence: 1.000
  Min confidence:     1.000
  Max confidence:     1.000
  Total questions:    5

Borderline Questions:
  Average confidence: 0.920
  Min confidence:     0.900
  Max confidence:     1.000
  Total questions:    5

📌 Confidence Interpretation:
  0.9 - 1.0: Very confident (clear classification)
  0.7 - 0.9: Confident (good classification)
  0.5 - 0.7: Moderate (borderline case)
  Below 0.5: Low (ambiguous question)


---
## Summary

Router Agent test results and observations.


In [10]:
print("📋 Router Agent Test Summary")
print("=" * 70)
print("\n✅ What We Tested:")
print("   • Clear NEEDLE questions (5 tests)")
print("   • Clear SUMMARY questions (5 tests)")
print("   • Borderline/edge cases (5 tests)")
print("   • Batch processing")
print("   • Confidence scoring")
print("\n✅ Router Agent Capabilities:")
print("   • Classifies questions into needle or summary routes")
print("   • Provides confidence scores (0.0 - 1.0)")
print("   • Explains reasoning for each decision")
print("   • Handles batch processing")
print("   • Deterministic (temperature=0.0)")
print("\n✅ What Router Agent Does NOT Do:")
print("   • Does NOT retrieve data from FAISS")
print("   • Does NOT access claim documents")
print("   • Does NOT generate final answers")
print("   • Does NOT use embeddings")
print("   • Does NOT know about specific claims")
print("\n🎯 Router Agent Role:")
print("   Classification ONLY - decides WHERE to route questions")
print("   Downstream agents handle retrieval and generation")
print("\n" + "=" * 70)
print("🎉 ALL TESTS COMPLETE!")
print("✅ Router Agent is ready for integration with retrieval agents")


📋 Router Agent Test Summary

✅ What We Tested:
   • Clear NEEDLE questions (5 tests)
   • Clear SUMMARY questions (5 tests)
   • Borderline/edge cases (5 tests)
   • Batch processing
   • Confidence scoring

✅ Router Agent Capabilities:
   • Classifies questions into needle or summary routes
   • Provides confidence scores (0.0 - 1.0)
   • Explains reasoning for each decision
   • Handles batch processing
   • Deterministic (temperature=0.0)

✅ What Router Agent Does NOT Do:
   • Does NOT retrieve data from FAISS
   • Does NOT access claim documents
   • Does NOT generate final answers
   • Does NOT use embeddings
   • Does NOT know about specific claims

🎯 Router Agent Role:
   Classification ONLY - decides WHERE to route questions
   Downstream agents handle retrieval and generation

🎉 ALL TESTS COMPLETE!
✅ Router Agent is ready for integration with retrieval agents


---
## Next Steps

The Router Agent is validated and ready.

### What's Ready:
✅ Question classification (needle vs summary)  
✅ Confidence scoring  
✅ Structured output (JSON)  
✅ Batch processing  
✅ Deterministic routing  

### Next: Claim Agent (Retrieval + Generation)
- Integrate Index Layer retrievers
- Use Router Agent decisions
- Implement needle retrieval flow
- Implement summary retrieval flow
- Add LLM generation
- Test end-to-end QA
